# Sanity
Okay this is mostly to help me keep my sanity.  I'm grabbing what I think is the most important parts of the helix notebook and trying to get a clean environment to build out what I need for this week.

In [1]:
import numpy as np
from AutoEncoders import GRAEAnchor
import pandas as pd
from data_creation import generate_line_and_helix, generate_line_and_helix_with_noise
from helpers import *
from mashspud import MASH, SPUD
from cluster_help import *

def mod_anchors(anchors, removal):
    sort_anchors=np.sort(anchors)
    sort_remove=list(removal)
    for x in range(len(sort_anchors)):
        for y in range(len(sort_remove)):
            if sort_anchors[x]>=sort_remove[y]:
                sort_anchors[x]=sort_anchors[x]-1
    return(np.sort(sort_anchors))

In [ ]:
line, helix = generate_line_and_helix(200)
line_noisy, helix_noisy,noise_points = generate_line_and_helix_with_noise(200, noise_percentage=0.10, noise_scale=0.35)

training=helix
training_line=line
testing=helix_noisy
testing_line=line_noisy
anchors = create_anchors(len(helix))[:10]

spud_helix = SPUD()
spud_helix.fit(training.to_numpy(), training_line.to_numpy(), anchors)
spud_helix.plot_emb(labels=None)
spud_noisy = SPUD()
spud_noisy.fit(testing.to_numpy(), testing_line.to_numpy(), anchors)
spud_noisy.plot_emb(labels=None)
mash_helix = MASH()
mash_helix.fit(training.to_numpy(), training_line.to_numpy(), anchors)
mash_helix.plot_emb(labels=None)
mash_noisy = MASH()
mash_noisy.fit(testing.to_numpy(), testing_line.to_numpy(), anchors)
mash_noisy.plot_emb(labels=None)
# print(type(mash_helix))

fig = plt.figure(figsize=(11, 7))
ax1 = fig.add_subplot(121)
ax1.plot(line_noisy['x'], line_noisy['y'], label='Line (noisy)', color='tab:cyan')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Line (noisy)')
ax1.legend()
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot(helix_noisy['x'], helix_noisy['y'], helix_noisy['z'], label='Helix (noisy)', color='tab:purple')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('z')
ax2.set_title('Helix (noisy)')
ax2.legend()
plt.tight_layout()
plt.show()

In [4]:
# Fitting the auto encoders.
AutoEncA = GRAEAnchor(lam=100, anchor_lam=100, relax=True, n_components = 2)
AutoEncA.fit(training.to_numpy(), spud_helix.emb[:len(training)], anchors)
AutoEncB = GRAEAnchor(lam=100, anchor_lam=100, relax=True, n_components = 2)
AutoEncB.fit(training_line.to_numpy(), spud_helix.emb[len(training_line):], anchors)
AutoEncA_m = GRAEAnchor(lam=100, anchor_lam=100, relax=False, n_components = 2)
AutoEncA_m.fit(training.to_numpy(), mash_helix.emb[len(training):], anchors)
AutoEncB_m = GRAEAnchor(lam=100, anchor_lam=100, relax=False, n_components = 2)
AutoEncB_m.fit(training_line.to_numpy(), mash_helix.emb[len(training):], anchors)
AutoEnc_snh = GRAEAnchor(lam=100, anchor_lam=100, relax=False, n_components = 2)
# Noisy stuff starts here.
AutoEnc_snh.fit(testing.to_numpy(), spud_noisy.emb[:len(testing)], anchors)
AutoEnc_snl = GRAEAnchor(lam=100, anchor_lam=100, relax=False, n_components = 2)
AutoEnc_snl.fit(testing_line.to_numpy(), spud_noisy.emb[len(testing):], anchors)
AutoEnc_mnh = GRAEAnchor(lam=100, anchor_lam=100, relax=False, n_components = 2)
AutoEnc_mnh.fit(testing.to_numpy(), mash_noisy.emb[:len(testing)], anchors)
AutoEnc_mnl = GRAEAnchor(lam=100, anchor_lam=100, relax=False, n_components = 2)
AutoEnc_mnl.fit(testing_line.to_numpy(), mash_noisy.emb[:len(testing_line)], anchors)

In [6]:
# Transformations and inverse transformations for noisy helix with SPUD and MASH values.
spud_helix_encoded=AutoEnc_snh.transform(helix_noisy.values)
mash_helix_encoded=AutoEnc_mnh.transform(helix_noisy.values)

inverter_spud=AutoEnc_snh.inverse_transform(spud_helix_encoded)
inverter_mash=AutoEnc_mnh.inverse_transform(mash_helix_encoded)

mash_line_encoded=AutoEnc_mnl.transform(line_noisy.values)
inverter_line=AutoEnc_mnl.inverse_transform(mash_line_encoded)

spud_line_encoder=AutoEnc_snl.transform(line_noisy.values)
spud_line_inverter=AutoEnc_snl.inverse_transform(spud_line_encoder)